In [77]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Debugging and Optimizing Agents: A Guide to Tracing in Reasoning Engine

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tracing_agents_in_reasoning_engine">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Freasoning-engine/tracing_agents_in_reasoning_engine.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/reasoning-engine/tracing_agents_in_reasoning_engine.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tracing_agents_in_reasoning_engine.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | |
|-|-|
|Author(s) | [Kristopher Overholt](https://github.com/koverholt)

## Overview

[Reasoning Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/overview) (LangChain on Vertex AI) is a managed service that helps you to build and deploy an agent reasoning framework. [LangGraph](https://langchain-ai.github.io/langgraph/) is a library for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows.

## TODO

- Update the below info for tracing, LangChain, and Cloud Trace
- Add narrative text throughout example / tutorial sections
- Switch tools to a different scenario

This notebook demonstrates how to use 

Note that the approach used in this notebook defines a [custom application template in Reasoning Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/customize), which can be extended to LangChain or other orchestration frameworks. If just want to use LangChain on Vertex AI to build agentic generative AI applications, refer to the documentation for [developing with the LangChain template in Reasoning Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/develop).

This notebook covers the following steps:

- **Define Tools**: Create custom Python functions to act as tools your AI application can use.
- **Define Router**: Create custom Python functions to act as tools your AI application can use.
- **Build a LangGraph Application**: Structure your application using LangGraph, including the Gemini model and custom tools that you define.
- **Local Testing**: Test your LangGraph application locally to ensure functionality.
- **Deploying to Vertex AI**: Seamlessly deploy your LangGraph application to Reasoning Engine for scalable execution.
- **Remote Testing**: Interact with your deployed application through Vertex AI, testing its functionality in a production-like environment.
- **Cleaning Up Resources**: Delete your deployed application on Vertex AI to avoid incurring unnecessary charges.

By the end of this notebook, you'll have the skills and knowledge to build and deploy your own custom generative AI applications using LangGraph, Reasoning Engine, and Vertex AI.

## Concepts

### Example trace

```json
{
   "name": "llm",
   "context": {
       "trace_id": "ed7b336d-e71a-46f0-a334-5f2e87cb6cfc",
       "span_id": "ad67332a-38bd-428e-9f62-538ba2fa90d4"
   },
   "span_kind": "LLM",
   "parent_id": "f89ebb7c-10f6-4bf8-8a74-57324d2556ef",
   "start_time": "2023-09-07T12:54:47.597121-06:00",
   "end_time": "2023-09-07T12:54:49.321811-06:00",
   "status_code": "OK",
   "status_message": "",
   "attributes": {
       "llm.input_messages": [
           {
               "message.role": "system",
               "message.content": "You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."
           },
           {
               "message.role": "user",
               "message.content": "Hello?"
           }
       ],
       "output.value": "assistant: Yes I am here",
       "output.mime_type": "text/plain"
   },
   "events": [],
}
```

### Trace

You can think of a [trace](https://opentelemetry.io/docs/concepts/signals/traces/) like a timeline of requests as they travel through your application. A trace is composed of individual spans, with the first span representing the overall request. Each span provides details about a specific operation within the request.

### Span

A [span](https://opentelemetry.io/docs/concepts/signals/traces/#spans) represents a single unit of work, like a function call or an interaction with an LLM. It captures information such as the operation's name, start and end times, and any relevant attributes (metadata). Spans can be nested, showing parent-child relationships between operations.

### Span Attribute

[Span attributes](https://opentelemetry.io/docs/concepts/signals/traces/#attributes) are key-value pairs that provide additional context about a span. For instance, an LLM span might have attributes like the model name, prompt text, and token count.

### Span Kind

[Span kind](https://opentelemetry.io/docs/concepts/signals/traces/#span-kind) categorizes the type of operation a span represents. Common kinds include:

- `CHAIN`: Links between LLM application steps or the start of a request.
- `LLM`: A call to a large language model.
- `TOOL`: An interaction with an external tool (API, database, etc.).
- `AGENT`: A reasoning block that combines LLM and tool interactions.

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [10]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
STAGING_BUCKET = "gs://[your-staging-bucket]"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

## Get started

### Install Vertex AI SDK and other required packages


In [14]:
%pip install --upgrade --user --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    google-cloud-trace

Note: you may need to restart the kernel to use updated packages.


### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


## Build and deploy an agent

### Import libraries

In [69]:
import pandas as pd
from google.cloud import trace_v1
from vertexai.preview import reasoning_engines
from vertexai.reasoning_engines._reasoning_engines import _utils

### Define tools

In [4]:
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date.

    Uses the Frankfurter API (https://api.frankfurter.app/) to obtain exchange rate data.

    Args:
        currency_from: The base currency (3-letter currency code). Defaults to "USD" (US Dollar).
        currency_to: The target currency (3-letter currency code). Defaults to "EUR" (Euro).
        currency_date: The date for which to retrieve the exchange rate. Defaults to "latest" for the most recent exchange rate data. Can be specified in YYYY-MM-DD format for historical rates.

    Returns:
        dict: A dictionary containing the exchange rate information.
             Example: {"amount": 1.0, "base": "USD", "date": "2023-11-24", "rates": {"EUR": 0.95534}}
    """
    import requests
    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

### Define agent and enable tracing

In [7]:
agent = reasoning_engines.LangchainAgent(
    model="gemini-1.5-pro-001",
    tools=[get_exchange_rate],
    enable_tracing=True,
    config={"metadata": {"session": "debug-123"},  # Optional tags to inject into traces
)

### Test your agent locally (with traces!)

In [33]:
agent.query(
    input="What's the exchange rate from US dollars to Swedish currency today?",
)

{'input': "What's the exchange rate from US dollars to Swedish currency today?",
 'output': 'The exchange rate from US dollars to Swedish krona is 1 USD to 10.61 SEK. \n'}

Even when you're testing your agent locally, you can view the traces for your agent to see details about X, Y, and Z.

To view the trace for the query that you just ran:

### Get your first trace

In [42]:
client = trace_v1.TraceServiceClient()

In [35]:
traces = [
    r for r in client.list_traces(request=trace_v1.types.ListTracesRequest(
        project_id=PROJECT_ID,
    ))
]

In [46]:
trace = traces[2]

In [47]:
trace

project_id: "koverholt-devrel-355716"
trace_id: "09bf2fa13718976563f1a41cdb5a6359"

In [65]:
traces = [r for r in client.list_traces(request=trace_v1.types.ListTracesRequest(
    project_id=PROJECT_ID,
    # Return all traces containing `labels {key: "openinference.span.kind" value: "AGENT"}`
    filter="openinference.span.kind:AGENT"
))]

In [66]:
trace = client.get_trace(project_id=PROJECT_ID, trace_id=traces[0].trace_id)

In [67]:
trace.spans[0]

span_id: 11469215384998756718
name: "AgentExecutor"
start_time {
  seconds: 1719610540
  nanos: 944411136
}
end_time {
  seconds: 1719610549
  nanos: 372617984
}
labels {
  key: "output.value"
  value: "The exchange rate from US dollars to Swedish krona is 1 USD to 10.61 SEK. \n"
}
labels {
  key: "openinference.span.kind"
  value: "AGENT"
}
labels {
  key: "input.value"
  value: "What\'s the exchange rate from US dollars to Swedish currency today?"
}
labels {
  key: "g.co/agent"
  value: "opentelemetry-python 1.25.0; google-cloud-trace-exporter 1.6.0"
}

After you deploy your agent and make remote queries in the following sections, you'll dive into the details for working with trace data in the Cloud Console or using the Python SDK for Cloud Trace.

### Deploy your agent

In [13]:
remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform[langchain,reasoningengine]",
        "cloudpickle==3.0.0",
        "pydantic==2.7.4",
    ],
)

Using bucket koverholt-devrel-355716-bucket
Writing to gs://koverholt-devrel-355716-bucket/reasoning_engine/reasoning_engine.pkl
Writing to gs://koverholt-devrel-355716-bucket/reasoning_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://koverholt-devrel-355716-bucket/reasoning_engine/dependencies.tar.gz
Creating ReasoningEngine
Create ReasoningEngine backing LRO: projects/964731510884/locations/us-central1/reasoningEngines/1586287415624990720/operations/1605235058988285952
ReasoningEngine created. Resource name: projects/964731510884/locations/us-central1/reasoningEngines/1586287415624990720
To use this ReasoningEngine in another session:
reasoning_engine = vertexai.preview.reasoning_engines.ReasoningEngine('projects/964731510884/locations/us-central1/reasoningEngines/1586287415624990720')


### Query your deployed agent

In [ ]:
remote_agent.query(
    input="What's the exchange rate from US dollars to Swedish currency?",
)

{'input': "What's the exchange rate from US dollars to Swedish currency?",
 'output': 'The exchange rate is 1 US Dollar to 10.4052 Swedish Krona.'}

## Exploring traces in the Cloud Console

https://cloud.google.com/trace/docs/finding-traces

1. For the project `gcp-project`, you can explore the traces using the Cloud Trace Explorer at https://console.cloud.google.com/traces/list?project=gcp-project

2. If you have the Trace ID (e.g. `your-trace-id`) for a particular trace, you can view it at https://console.cloud.google.com/traces/list?project=gcp-project&tid=abcd1234

## Working with traces using `pandas`

In [70]:
spans = pd.DataFrame.from_records([_utils.to_dict(span) for span in trace.spans])
spans

,spanId,name,startTime,endTime,labels,parentSpanId
0,11469215384998756718,AgentExecutor,2024-06-28T21:35:40.944411136Z,2024-06-28T21:35:49.372617984Z,{'g.co/agent': 'opentelemetry-python 1.25.0; g...,NaN
1,1605890896725060040,RunnableSequence,2024-06-28T21:35:40.955333120Z,2024-06-28T21:35:44.654963968Z,{'g.co/agent': 'opentelemetry-python 1.25.0; g...,11469215384998756718
2,5360685900019806995,get_exchange_rate,2024-06-28T21:35:44.869442816Z,2024-06-28T21:35:45.742665984Z,"{'tool.name': 'get_exchange_rate', 'g.co/agent...",11469215384998756718
3,16201913772494123770,"RunnableParallel<input,agent_scratchpad>",2024-06-28T21:35:45.973032960Z,2024-06-28T21:35:46.231666176Z,{'g.co/agent': 'opentelemetry-python 1.25.0; g...,16869047450359710008
4,12176265709676253650,ChatPromptTemplate,2024-06-28T21:35:46.417494016Z,2024-06-28T21:35:46.418998016Z,{'g.co/agent': 'opentelemetry-python 1.25.0; g...,16869047450359710008
5,13447144205799117434,"RunnableParallel<input,agent_scratchpad>",2024-06-28T21:35:40.957278976Z,2024-06-28T21:35:41.509939968Z,{'g.co/agent': 'opentelemetry-python 1.25.0; g...,1605890896725060040
6,12949462071645837552,RunnableLambda,2024-06-28T21:35:40.957742848Z,2024-06-28T21:35:40.958074112Z,{'g.co/agent': 'opentelemetry-python 1.25.0; g...,13447144205799117434
7,5639600538414318182,RunnableLambda,2024-06-28T21:35:40.959088128Z,2024-06-28T21:35:40.959360Z,{'g.co/agent': 'opentelemetry-python 1.25.0; g...,13447144205799117434
8,10647705944622325788,ChatPromptTemplate,2024-06-28T21:35:41.793710080Z,2024-06-28T21:35:41.795612160Z,{'g.co/agent': 'opentelemetry-python 1.25.0; g...,1605890896725060040
9,8581611416918654378,ChatVertexAI,2024-06-28T21:35:42.034980096Z,2024-06-28T21:35:44.138663936Z,{'g.co/agent': 'opentelemetry-python 1.25.0; g...,1605890896725060040


In [71]:
spans[spans["name"] == "ChatVertexAI"]

,spanId,name,startTime,endTime,labels,parentSpanId
9,8581611416918654378,ChatVertexAI,2024-06-28T21:35:42.034980096Z,2024-06-28T21:35:44.138663936Z,{'g.co/agent': 'opentelemetry-python 1.25.0; g...,1605890896725060040
14,1664264463028383759,ChatVertexAI,2024-06-28T21:35:46.753844992Z,2024-06-28T21:35:48.858014208Z,{'llm.input_messages.1.message.role': 'assistant...,16869047450359710008


In [72]:
spans[spans["name"] == "ChatVertexAI"].labels.apply(pd.Series)

,g.co/agent,llm.invocation_parameters,output.mime_type,llm.output_messages.0.message.function_call_arguments_json,llm.input_messages.0.message.content,output.value,llm.input_messages.0.message.role,input.mime_type,input.value,llm.output_messages.0.message.function_call_name,openinference.span.kind,llm.model_name,llm.function_call,llm.output_messages.0.message.role,llm.input_messages.1.message.role,llm.input_messages.2.message.role,llm.input_messages.1.message.function_call_arguments_json,llm.input_messages.2.message.content,llm.output_messages.0.message.content,llm.input_messages.1.message.function_call_name
9,opentelemetry-python 1.25.0; google-cloud-trac...,"{""model_name"": ""gemini-1.5-pro-001"", ""candidate...",application/json,"{""currency_from"": ""USD"", ""currency_to"": ""SEK""}",What's the exchange rate from US dollars to Sw...,"{""generations"": [[{""text"": """", ""generation_inf...",user,application/json,"{""messages"": [[{""lc"": 1, ""type"": ""constructor""...",get_exchange_rate,LLM,gemini-1.5-pro-001,"{""name"": ""get_exchange_rate"", ""arguments"": {""c...",assistant,NaN,NaN,NaN,NaN,NaN,NaN
14,opentelemetry-python 1.25.0; google-cloud-trac...,"{""model_name"": ""gemini-1.5-pro-001"", ""candidate...",application/json,NaN,What's the exchange rate from US dollars to Sw...,"{""generations"": [[{""text"": ""The exchange rate ...",user,application/json,"{""messages"": [[{""lc"": 1, ""type"": ""constructor""...",NaN,LLM,gemini-1.5-pro-001,NaN,assistant,assistant,tool,"{""currency_from"": ""USD"", ""currency_to"": ""SEK""}","{""amount"": 1.0, ""base"": ""USD"", ""date"": ""2024-0...",The exchange rate from US dollars to Swedish k...,get_exchange_rate


## Exploring traces with the Python SDK for Cloud Trace

### Filter by date and time

In [ ]:
result = client.list_traces(request=trace_v1.types.ListTracesRequest(
    project_id=PROJECT_ID,
    start_time="2024-06-10T00:00:00Z", # optional
    end_time="2024-06-14T23:59:59Z", # optional
))

for r in result:
    print(r)

### Filter by label

In [ ]:
result = client.list_traces(request=trace_v1.types.ListTracesRequest(
    project_id=PROJECT_ID,
    # Return traces where any root span's name starts with AgentExecutor
    filter="root:AgentExecutor"
)):

for r in result:
    print(r)

### Filter by view type

In [ ]:
for r in client.list_traces(request=trace_v1.types.ListTracesRequest(
    project_id=PROJECT_ID,
    view=trace_v1.types.ListTracesRequest.ViewType.ROOTSPAN,
    # view=trace_v1.types.ListTracesRequest.ViewType.MINIMAL,
    # view=trace_v1.types.ListTracesRequest.ViewType.COMPLETE,
)):
    print(r)

In [ ]:
trace = client.get_trace(project_id=PROJECT_ID, trace_id="9dd0c06e88bda06b197b5eff0409e8f9")

trace.project_id, trace.trace_id

traces = [
    r for r in client.list_traces(request=trace_v1.types.ListTracesRequest(
        project_id=PROJECT_ID,
        start_time="2024-06-10T00:00:00Z",
        end_time="2024-06-14T23:59:59Z",
        view=trace_v1.types.ListTracesRequest.ViewType.COMPLETE,
    ))
]

traces[0] # Show just the first one

## Cleaning up

In [ ]:
remote_app.delete()